# Backtesting.pyライブラリによるバックテスト

In [1]:
import pandas as pd
import yfinance as yf
import talib as ta
from datetime import datetime, timedelta
import numpy as np
from pandas_datareader import data as pdr
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG, EURUSD
import warnings #警告をコントロールするライブラリを追加
warnings.filterwarnings("ignore") #警告を非表示に設定


C:\Users\nicjp\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\nicjp\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\nicjp\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:
# 結果を別画面に表示させる
#from backtesting import set_bokeh_output
#set_bokeh_output(notebook=False)


## 株価データの取得

In [3]:
# Define the stock symbol and timeframe
symbol = '9432.T' # 9432はNTT
end_date = datetime(2023, 9, 30, 0, 0, 0)
start_date = end_date - timedelta(days=1800)  # 5 years before today
yf.pdr_override()
# yahooサイトからデータをダウンロード
stock_data = pdr.get_data_yahoo(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


## 取引戦略のクラスを作成する（Strategyを継承）

In [4]:
class TalibCross(Strategy):
    tp1 = 10
    tp2 = 20
    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.SMA, close, self.tp1)
        self.sma2 = self.I(ta.SMA, close, self.tp2)
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()        


## Backtestインスタンスの生成

In [5]:
bt = Backtest(stock_data, TalibCross, cash=1000000)

## バックテストの実行

In [6]:
output = bt.run()
print(output)

Start                     2018-10-26 00:00:00
End                       2023-09-29 00:00:00
Duration                   1799 days 00:00:00
Exposure Time [%]                   62.427265
Equity Final [$]               1323264.751915
Equity Peak [$]                1348285.306999
Return [%]                          32.326475
Buy & Hold Return [%]               80.572598
Return (Ann.) [%]                    6.043026
Volatility (Ann.) [%]               15.365342
Sharpe Ratio                         0.393289
Sortino Ratio                        0.624752
Calmar Ratio                         0.356545
Max. Drawdown [%]                  -16.948859
Avg. Drawdown [%]                   -2.610368
Max. Drawdown Duration      841 days 00:00:00
Avg. Drawdown Duration       53 days 00:00:00
# Trades                                   34
Win Rate [%]                        44.117647
Best Trade [%]                      13.421826
Worst Trade [%]                     -8.269959
Avg. Trade [%]                    

In [7]:
bt.plot()

GridPlot(id='p1309', ...)

## 最適化

In [8]:
output = bt.optimize(
    tp1 = range(4, 20, 1),
    tp2 = range(14, 30, 1),
    constraint=lambda p: p.tp1 < p.tp2,
    maximize = 'Sharpe Ratio')
print(output)

  0%|          | 0/17 [00:00<?, ?it/s]

Start                     2018-10-26 00:00:00
End                       2023-09-29 00:00:00
Duration                   1799 days 00:00:00
Exposure Time [%]                   62.177889
Equity Final [$]               1790243.594528
Equity Peak [$]                1824093.933762
Return [%]                          79.024359
Buy & Hold Return [%]               80.572598
Return (Ann.) [%]                   12.974154
Volatility (Ann.) [%]               15.913723
Sharpe Ratio                         0.815281
Sortino Ratio                        1.492654
Calmar Ratio                         1.158044
Max. Drawdown [%]                  -11.203509
Avg. Drawdown [%]                   -2.902265
Max. Drawdown Duration      398 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   47
Win Rate [%]                         51.06383
Best Trade [%]                      20.556791
Worst Trade [%]                     -4.577393
Avg. Trade [%]                    

## 最適化されたパラメータを確認する

In [9]:
print(output._strategy)

TalibCross(tp1=5,tp2=15)


In [10]:
#print(output._strategy.n1,output._strategy.n2)